In [3]:
PATH = "../../../../../KhoaData" 
!ls {PATH}

departments.csv  UEH-qa-1.json	 UEH-sub-2.json  UEH-sub-5.json  UE-qa-2.json
mk_ques.csv	 UEH-qa-3.json	 UEH-sub-3.json  UEH-sub-6.json
UEH-main.json	 UEH-sub-1.json  UEH-sub-4.json  UEH-sub-7.json


# TDT data

In [1]:
from datasets import load_dataset

QA = "BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong"
DS = "BroDeadlines/CRAWL.TDT.mini.dai-hoc"
# qa = load_dataset(QA)
# ds = load_dataset(DS, "compact")

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['url', 'html_name', 'skipped', 'text'],
        num_rows: 399
    })
})

### New copora

In [4]:
qa['INDEX.medium_index_TDT']

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 144
})

In [18]:
ads = ds.rename_column("text", "content")
ads = ads.map(lambda a: {**a, "metadata": ""})
ads = ads.remove_columns(['html_name', "skipped"])
ads

DatasetDict({
    train: Dataset({
        features: ['url', 'content', 'metadata'],
        num_rows: 399
    })
})

In [20]:
from uuid import uuid4
ads = ads.map(lambda e: {**e, "doc_id": str(uuid4())})
ads

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'content', 'metadata', 'doc_id'],
        num_rows: 399
    })
})

In [8]:
import json

def reformat(row):
    row['metadata'] = json.dumps({"isQA": True})
    return row
    
aqa = qa['INDEX.medium_index_TDT'].map(reformat)
aqa = aqa.remove_columns(['question', 'group'])
aqa = aqa.rename_column("answer", 'content')
aqa

Dataset({
    features: ['content', 'url', 'doc_id', 'metadata'],
    num_rows: 144
})

In [22]:
from datasets import concatenate_datasets

aads = concatenate_datasets([ads['train'], aqa])

aads

aads.push_to_hub("BroDeadlines/TEST.TDT.mini.tdt_copora_data", "compact")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/873 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.TDT.mini.tdt_copora_data/commit/c6ff13c3ebdda1d0041dc0f2eb59ed5e3ab3f59c', commit_message='Upload dataset', commit_description='', oid='c6ff13c3ebdda1d0041dc0f2eb59ed5e3ab3f59c', pr_url=None, pr_revision=None, pr_num=None)

## Remove to Compact

In [42]:
removes = ["enlabsafe", "clb", "tracuu", "hoc-bong", "ban-giam-hieu", 
           "hoi-dong", "don-vi", "so-lieu", "itc", "doi-nhom", "cham-dut", 
           "ChuyenDeTamLy", "hanh-chinh", "kiem-dinh", "thanh-tich", "student.tdtu.edu.vn/doi-", 
           "band", "kiem-dinh", "tuyen_dung", "aggregator", "fms", "law"]
filtered_dataset = ds
for r in removes: 
    filtered_dataset = filtered_dataset.filter(lambda e: r not in e['url'])
filtered_dataset

Filter:   0%|          | 0/422 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'html_name', 'skipped', 'text'],
        num_rows: 400
    })
})

In [43]:
filtered_dataset = filtered_dataset.filter(lambda e: "Javascript" not in e['text'])
filtered_dataset

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'html_name', 'skipped', 'text'],
        num_rows: 399
    })
})

In [44]:
import pandas as pd
from datasets import Dataset

df = pd.DataFrame(filtered_dataset['train'])
df = df.drop_duplicates(subset="url", keep='first')  # or 'last'
fin_dataset = Dataset.from_pandas(df)
fin_dataset

Dataset({
    features: ['url', 'html_name', 'skipped', 'text'],
    num_rows: 399
})

In [46]:
filtered_dataset

DatasetDict({
    train: Dataset({
        features: ['url', 'html_name', 'skipped', 'text'],
        num_rows: 399
    })
})

In [50]:
ds.push_to_hub(DS)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/901 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/CRAWL.TDT.mini.dai-hoc/commit/7d002cb223c64e87e44809887964c53d0df6e4d8', commit_message='Upload dataset', commit_description='', oid='7d002cb223c64e87e44809887964c53d0df6e4d8', pr_url=None, pr_revision=None, pr_num=None)

In [54]:
DS

'BroDeadlines/CRAWL.TDT.mini.dai-hoc'

In [55]:
filtered_dataset.push_to_hub('BroDeadlines/CRAWL.TDT.mini.compact.dai-hoc')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/CRAWL.TDT.mini.compact.dai-hoc/commit/fd8928cfd8ff968300ab9f66bbbcec2eca1b9e7e', commit_message='Upload dataset', commit_description='', oid='fd8928cfd8ff968300ab9f66bbbcec2eca1b9e7e', pr_url=None, pr_revision=None, pr_num=None)

# HUB data

In [2]:
import pandas as pd

df = pd.read_json("~/Downloads/ts_hub_records.jsonl", lines=True)

print(df.shape)

adf = df[df['content'].str.strip().str.len() > 0]
adf.shape

(4687, 4)


(4618, 4)

In [3]:
import re

def clean_txt(doc):
  strips = ["\n{3,}", "\s{3,}", "\t{3, }"]
  replaces = ["\n", " ", " "]
  for strip, replace in zip(strips, replaces):
      doc = re.sub(strip, replace, doc)
  return doc

contents = adf['content'].apply(clean_txt)
titles = adf['title'].apply(lambda e: e.strip())

In [4]:
adf['title'] = titles
adf['content'] = contents

/tmp/ipykernel_12913/3932715025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adf['title'] = titles
/tmp/ipykernel_12913/3932715025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adf['content'] = contents


In [5]:
df = adf.drop_duplicates()
df.shape

(4618, 4)

### Filter QA

In [11]:
KEY = "cau-hoi"

ddf = df[df['url'].str.contains(KEY)]
adf = df[~df['url'].str.contains(KEY)]

In [14]:
df = ddf
df.shape

In [16]:
choose = [".2024", "9.2023", "8.2023", "7.2023"]
CHOOSES = "|".join(choose)

ddf = df[df['content'].str.contains(CHOOSES)]
adf = df[~df['content'].str.contains(CHOOSES)]

In [17]:
ddf.shape

(104, 4)

In [22]:
fdf = ddf.iloc[1: , :]
fdf.shape

(103, 4)

In [29]:
fdf.columns

Index(['url', 'skipped', 'title', 'content'], dtype='object')

In [27]:
a = fdf.iloc[0]['content']
aa = a.split("Trả lời:")

aa

[' Nộp học bạ đầy đủ 3 năm là HK1, HK2 lớp 11 và HK1 lớp 12 đúng ko vậy ạ ',
 ' Chào em, Với phương thức xét tuyển tổng hợp (học bạ và thành tích bậc THPT) em nộp hồ sơ gồm: - Phiếu đăng ký xét tuyển (Sau khi đăng ký xét tuyển trực tuyến thành công, thí sinh in phiếu đăng ký và ký tên lên phiếu); -\xa0\nChứng minh nhân dân/Thẻ căn cước công dân (bản photo chứng thực); -\xa0\nChứng nhận đối tượng ưu tiên (bản photo chứng thực) (nếu có); - Chứng chỉ Tiếng Anh quốc tế (bản photo chứng thực) (nếu có); -\xa0\nGiấy chứng nhận đạt giải kỳ thi chọn học sinh giỏi, kỳ thi Olympic, Quốc gia, cấp tỉnh và thành phố trực t\nhuộc trung ương (bản photo chứng thực) (nếu có); - Giấy xác nhận điểm hoặc bản sao học bạ cụ thể như sau: +\xa0\nTrường hợp thí sinh chưa tốt nghiệp THPT\n: Nộp giấy xác nhận điểm có xác nhận của trường THPT (Mẫu giấy xác nhận điểm đính kèm tại thông báo).\xa0\nTrường sẽ yêu cầu thí sinh nộp Giấy chứng nhận tốt nghiệp tạm thời và học bạ đầy đủ 3 năm học THPT (bản photo chứng thực

In [37]:
QAs = []
def build_QA(row):
    d = row['content']
    dd = d.split("Trả lời:")
    if len(dd) == 1:
        dd = d.split("trả lời:")
    if len(dd) < 2:
        print(f"Error: {d}")
        return
    data = {
        "url": row['url'],
        "group": row['title'],
        "question": dd[0].replace("\xa0", "").strip(),
        "answer": dd[1].replace("\xa0", "").strip()
    }
    QAs.append(data)
    return data
    

In [44]:
# [build_QA(r) for idx, r in fdf.iterrows()]

len(QAs)

102

#### Swap

In [58]:
repo = "BroDeadlines/QA.HUB.QA_tu_van_tuyen_sinh"
from datasets import load_dataset

ds = load_dataset(repo)
ds

Generating train split:   0%|          | 0/102 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'group', 'question', 'answer'],
        num_rows: 102
    })
})

In [59]:
def swap(row):
    g = row['group']
    q = row['question']
    if len(g) > len(q):
        print("SWAP")
        row['question'] = g
        row['group'] = q
    return row

ads = ds.map(swap)
ads

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

SWAP


DatasetDict({
    train: Dataset({
        features: ['url', 'group', 'question', 'answer'],
        num_rows: 102
    })
})

In [60]:
ads.push_to_hub(repo)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/372 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/QA.HUB.QA_tu_van_tuyen_sinh/commit/8a463acd459a7dfab713dceae882674eb473f311', commit_message='Upload dataset', commit_description='', oid='8a463acd459a7dfab713dceae882674eb473f311', pr_url=None, pr_revision=None, pr_num=None)

### Re-clean

In [52]:
repo = "BroDeadlines/CRAWL.HUB.tuyensinh.edu"
repo = "BroDeadlines/CRAWL.HUB.mini.tuyensinh.edu"
from datasets import load_dataset

ds = load_dataset(repo)
ds

Generating train split:   0%|          | 0/316 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'skipped', 'title', 'content', '__index_level_0__'],
        num_rows: 316
    })
})

In [53]:
ds = ds.remove_columns(["__index_level_0__"])
ds

DatasetDict({
    train: Dataset({
        features: ['url', 'skipped', 'title', 'content'],
        num_rows: 316
    })
})

In [54]:
def clean(row):
    defect = "\xa0"
    row['title'] = row['title'].replace(defect,"").strip()
    row['content'] = row['content'].replace(defect,"").strip()
    row['isClean'] = True
    return row

In [55]:
ads = ds.map(clean)

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

In [56]:
ads = ads.remove_columns(["isClean"])
ads

DatasetDict({
    train: Dataset({
        features: ['url', 'skipped', 'title', 'content'],
        num_rows: 316
    })
})

In [57]:
print(repo)
ads.push_to_hub(repo)

BroDeadlines/CRAWL.HUB.mini.tuyensinh.edu


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/422 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/CRAWL.HUB.mini.tuyensinh.edu/commit/15192f618ba0267d0a9ab7eada52c96455e49fa2', commit_message='Upload dataset', commit_description='', oid='15192f618ba0267d0a9ab7eada52c96455e49fa2', pr_url=None, pr_revision=None, pr_num=None)

### Text copora

In [1]:
from uuid import uuid4
from datasets import load_dataset
qa_repo = "BroDeadlines/QA.HUB.QA_tu_van_tuyen_sinh"
da_repo = "BroDeadlines/CRAWL.HUB.mini.tuyensinh.edu"

qa_ds = load_dataset(qa_repo)
da_ds = load_dataset(da_repo)

print(qa_ds)
da_ds

Generating train split:   0%|          | 0/102 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/316 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'group', 'question', 'answer'],
        num_rows: 102
    })
})


DatasetDict({
    train: Dataset({
        features: ['url', 'skipped', 'title', 'content'],
        num_rows: 316
    })
})

In [21]:
def qa_to_ds(row):
    data = {
        "title": row['group'],
        "url": row['url'],
        "doc_id": row['doc_id']
    }
    content = row['answer']
    removes = ["Chào em,", "Chào em!", "Chào em"]
    for r in removes:
        if r in content:
            content = content.replace(r, "").strip()
        if "\xa0" in content:
            print("ERROR")
    data['content'] = content
    return data

In [22]:
print(qa_ds)
da_ds

DatasetDict({
    train: Dataset({
        features: ['url', 'group', 'question', 'answer', 'doc_id'],
        num_rows: 102
    })
})


DatasetDict({
    train: Dataset({
        features: ['url', 'skipped', 'title', 'content', 'doc_id'],
        num_rows: 316
    })
})

In [29]:
qa_ads = qa_ds.map(qa_to_ds)
qa_ads = qa_ads.remove_columns(['group', 'question', 'answer'])
qa_ads

DatasetDict({
    train: Dataset({
        features: ['url', 'doc_id', 'title', 'content'],
        num_rows: 102
    })
})

In [31]:
da_ads = da_ds.remove_columns(['skipped'])
da_ads

DatasetDict({
    train: Dataset({
        features: ['url', 'title', 'content', 'doc_id'],
        num_rows: 316
    })
})

In [33]:
from datasets import concatenate_datasets
bds = concatenate_datasets([qa_ads['train'], da_ads['train']])

bds

Dataset({
    features: ['url', 'doc_id', 'title', 'content'],
    num_rows: 418
})

In [37]:
shuf_bds = bds.shuffle(29)

shuf_bds.push_to_hub("BroDeadlines/TEST.HUB.mini.hub_copora_data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.HUB.mini.hub_copora_data/commit/616ecbb3c2e4d2e39a0918f66d2d113e545288ff', commit_message='Upload dataset', commit_description='', oid='616ecbb3c2e4d2e39a0918f66d2d113e545288ff', pr_url=None, pr_revision=None, pr_num=None)

### Push

In [39]:
from datasets import Dataset
print(len(QAs))
ds = Dataset.from_list(QAs)

ds

102


Dataset({
    features: ['url', 'group', 'question', 'answer'],
    num_rows: 102
})

In [40]:
repo = "BroDeadlines/QA.HUB.QA_tu_van_tuyen_sinh"

ds.push_to_hub(repo)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/QA.HUB.QA_tu_van_tuyen_sinh/commit/66914c1de91e2cf307b03949fafbbb4c33c5823a', commit_message='Upload dataset', commit_description='', oid='66914c1de91e2cf307b03949fafbbb4c33c5823a', pr_url=None, pr_revision=None, pr_num=None)

In [7]:
print(f"Old:{df.shape}")
adf.shape

Old:(4618, 4)


(316, 4)

In [ ]:
adf = adf[adf['url'].str.contains("")]

In [30]:
# adf = adf.drop(['__index_level_0__'])
minids = Dataset.from_pandas(adf)
minids

Dataset({
    features: ['url', 'skipped', 'title', 'content', '__index_level_0__'],
    num_rows: 316
})

In [34]:
repo = "BroDeadlines/CRAWL.HUB.mini.tuyensinh.edu"

minids.push_to_hub(repo)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/CRAWL.HUB.mini.tuyensinh.edu/commit/e40753ee008fdb69b6e2a48c25aa7b59e8b924cb', commit_message='Upload dataset', commit_description='', oid='e40753ee008fdb69b6e2a48c25aa7b59e8b924cb', pr_url=None, pr_revision=None, pr_num=None)

# UEH data

In [2]:
files = ['UEH-main.json', *[f"UEH-sub-{i}.json" for i in range(1,8)]]
files

['UEH-main.json',
 'UEH-sub-1.json',
 'UEH-sub-2.json',
 'UEH-sub-3.json',
 'UEH-sub-4.json',
 'UEH-sub-5.json',
 'UEH-sub-6.json',
 'UEH-sub-7.json']

In [3]:
import pandas as pd
fin_df = None
for f in files:
    df = pd.read_json(f"{PATH}/{f}")
    print(df.shape)
    if fin_df is None:
        fin_df = df
    else:
        fin_df = pd.concat([fin_df, df])

(196, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)


In [4]:
REMOVES = ["CHỌN HỆ ĐÀO TẠO\nSkip to content\n", "n\nTin, ảnh, video: Phân hiệu Vĩnh Long\n\nTrang chủ\nTư vấn trước nhập học\nĐăng ký nhập học\nĐịnh hướng sau nhập học\nTư vấn Chương trình đào tạo\nTư vấn qua điện thoại, email\nTham quan UEH\nKhám phá UEH\nUEH Shop\nWe love UEH (video)\nNhịp sống sinh viên\n360 UEH\nUEH Green Campus\nBản tin UEH\nUEH Hotel\n\n© Copyright Đại học Kinh tế TP. Hồ Chí Minh"]


In [17]:
import re

def clean_txt(doc):
  strips = ["\n{3,}", "\s{3,}", "\t{3, }"]
  replaces = ["\n\n", "\n\n", " "]
  for strip, replace in zip(strips, replaces):
      doc = re.sub(strip, replace, doc)
  return doc

def clean(txt):
    for r in REMOVES:
        txt = txt.replace(r, "")
        txt = txt.replace(u'\xa0', u' ')
        txt = clean_txt(txt)
    return txt.strip()
    
clean_txts = fin_df['html'].apply(clean)

In [19]:
print(clean_txts.iloc[-1])

| BƯỚC 1
| BƯỚC 2
| BƯỚC 3
| BƯỚC 4
hòa nhập ueh

#gap-559077276 {

padding-top: 10px;
}

ĐÓNG LỆ PHÍ XÉT TUYỂN
Sinh viên đã thực hiện đóng lệ phí bỏ qua bước này.
Sinh viên thực hiện đóng lệ phí, hạn chót ngày 16/8/2024, theo 1 trong 2 cách sau:
Cách 1 (khuyến nghị áp dụng): 
– Truy cập link https://xettuyendaihoc.ueh.edu.vn/lttc-vlvh
– Chọn “Kê khai hồ sơ”; đăng nhập theo thông tin đã kê khai.
– Chọn “Nộp tiền Online”, nằm ở cuối trang, dòng tô màu xanh.
Cách 2
– Truy cập link go.ueh.edu.vn/phixettuyenVLVH
– Nhập “Mã hồ sơ” thí sinh đã được nhận qua email, bấm “Tìm”.
– Bấm “Tiếp tục”
– Bước 2: Đánh dấu (check) vào mục điều kiện thanh toán “Tôi đã đọc kỹ và tuân thủ…”
– Chọn Phương thức thanh toán, bấm “Payoo” để thực hiện thanh toán.

Mọi thắc mắc vui lòng gửi đến email tuyensinhvlvh@ueh.edu.vn

#gap-1625639550 {

padding-top: 10px;
}

Đóng học phí
Sinh viên thực hiện đóng học phí theo thời hạn đã thông tin trong thông báo trúng tuyển, thực hiện đóng học phí theo 1 trong 2 cách sau:


## Upload data

In [21]:
from datasets import Dataset

ds = Dataset.from_pandas(fin_df)

ds

Dataset({
    features: ['title', 'url', 'html', '__index_level_0__'],
    num_rows: 203
})

In [22]:
from uuid import uuid4

def format(row):
    txt = row['html']
    row['content'] = clean(txt)
    row['doc_id'] = str(uuid4())
    return row

ads = ds.map(format)

ads

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'url', 'html', '__index_level_0__', 'content', 'doc_id'],
    num_rows: 203
})

In [24]:
ads = ads.remove_columns(['__index_level_0__'])
ads

Dataset({
    features: ['title', 'url', 'html', 'content', 'doc_id'],
    num_rows: 203
})

In [25]:
ads.push_to_hub("BroDeadlines/CRAWL.UEH.tuyensinh.edu")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/CRAWL.UEH.tuyensinh.edu/commit/141c9a12a0ab022790770956a4928d2ee8dc6f08', commit_message='Upload dataset', commit_description='', oid='141c9a12a0ab022790770956a4928d2ee8dc6f08', pr_url=None, pr_revision=None, pr_num=None)

## QA data

In [39]:
import pandas as pd

fin_df = pd.read_csv(f"{PATH}/QA_UEH.csv")
fin_df.shape

(54, 4)

In [41]:
from datasets import Dataset

ds = Dataset.from_pandas(fin_df)
ds

Dataset({
    features: ['question', 'answer', 'url', 'group'],
    num_rows: 54
})

In [43]:
from uuid import uuid4

def format(row):
    row['question'] = row['question'].strip()
    row['answer'] = row['answer'].strip()
    row['doc_id'] = str(uuid4())
    return row

In [44]:
ads = ds.map(format)
ads

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id'],
    num_rows: 54
})

In [45]:
ads.push_to_hub("BroDeadlines/QA.UEH.QA_tu_van_tuyen_sinh")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/QA.UEH.QA_tu_van_tuyen_sinh/commit/71f0ced8dd491bbd88bc663546c557c6b38561f4', commit_message='Upload dataset', commit_description='', oid='71f0ced8dd491bbd88bc663546c557c6b38561f4', pr_url=None, pr_revision=None, pr_num=None)

## COpora

In [3]:
repo1 = "BroDeadlines/CRAWL.UEH.tuyensinh.edu"
repo2 = "BroDeadlines/QA.UEH.QA_tu_van_tuyen_sinh"
ds1 = load_dataset(repo1)
ds2 = load_dataset(repo2)

Generating train split:   0%|          | 0/203 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/54 [00:00<?, ? examples/s]

In [8]:
ads2 = ds2.remove_columns(['question', 'group'])

ads2 = ads2.rename_column("answer", "content")

ads2

DatasetDict({
    train: Dataset({
        features: ['content', 'url', 'doc_id'],
        num_rows: 54
    })
})

In [18]:
import json
ads2 = ads2.map(lambda e: {**e, "metadata":json.dumps({"isQA": True})})
ads2

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'url', 'doc_id', 'metadata'],
        num_rows: 54
    })
})

In [16]:
ads1 = ds1.remove_columns(['html', 'title'])
ads1 = ads1.map(lambda e: {**e, "metadata": ""})
ads1

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'content', 'doc_id', 'metadata'],
        num_rows: 203
    })
})

In [21]:
from datasets import concatenate_datasets

fads = concatenate_datasets([ads1['train'], ads2['train']])
fads

Dataset({
    features: ['url', 'content', 'doc_id', 'metadata'],
    num_rows: 257
})

In [22]:
fads.push_to_hub("BroDeadlines/TEST.UEH.ueh_copora_data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.UEH.ueh_copora_data/commit/55e918f87ec7b99d7484aa85b4ab0825c3ce44bb', commit_message='Upload dataset', commit_description='', oid='55e918f87ec7b99d7484aa85b4ab0825c3ce44bb', pr_url=None, pr_revision=None, pr_num=None)

# TDT

## Build text-copora

In [1]:
from datasets import load_dataset

main_ds = load_dataset("BroDeadlines/CRAWL.TDT.mini.dai-hoc")
main_ds

Generating train split:   0%|          | 0/597 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'html_name', 'skipped', 'text'],
        num_rows: 597
    })
})

In [2]:
eval_ds = load_dataset("BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong")
eval_ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
        num_rows: 215
    })
    TEST.basic_test_tdt_dataset: Dataset({
        features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
        num_rows: 20
    })
    INDEX.medium_index_TDT: Dataset({
        features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
        num_rows: 144
    })
})

In [3]:
import json
def notClone(row):
    d = row['metadata']
    if len(d) == 0:
        return True
    d = json.loads(d)
    return not d['isCloned']
    
filter_ds = eval_ds['train'].filter(notClone)
filter_ds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 185
})

In [4]:
from uuid import uuid4
IDS = [str(uuid4()) for i in range(2995)]

def mini(e):
    d = {"url": e['url'], "content": e['text'], "doc_id": IDS.pop(), "metadata": ""}
    return d

main_ds_one = main_ds.map(mini)
main_ds_one

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'html_name', 'skipped', 'text', 'content', 'doc_id', 'metadata'],
        num_rows: 597
    })
})

In [6]:
main_ds_two = main_ds_one.remove_columns(['html_name', "skipped", "text"])
main_ds_two

DatasetDict({
    train: Dataset({
        features: ['url', 'content', 'doc_id', 'metadata'],
        num_rows: 597
    })
})

In [7]:
filter_ds_one = filter_ds.remove_columns(['question', 'group']).rename_column("answer", "content")
filter_ds_one

Dataset({
    features: ['content', 'url', 'doc_id', 'metadata'],
    num_rows: 185
})

In [8]:
import json

filter_ds_one = filter_ds_one.map(lambda e: {"metadata": json.dumps({"isQA": True})})

filter_ds_one

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'url', 'doc_id', 'metadata'],
    num_rows: 185
})

In [35]:
filter_ds_one[0]

{'text': 'Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.',
 'url': 'https://tuvanhocduong.tdtu.edu.vn/News',
 'doc_id': '573ca174-1b6d-11ef-a755-d38426455a06',
 'metadata': '{"isQA": true}'}

In [10]:
from datasets import concatenate_datasets
index_ds = concatenate_datasets([main_ds_two['train'],filter_ds_one])

index_ds_one = index_ds.filter(lambda e: "javascript" not in e['content'].lower())
index_ds_one

Filter:   0%|          | 0/782 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'content', 'doc_id', 'metadata'],
    num_rows: 781
})

In [11]:
def check_QA(row):
    d = row['metadata']
    if d == "":
        return False
    d = json.loads(d)
    return d['isQA']
index_ds_one.filter(check_QA)

Filter:   0%|          | 0/781 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'content', 'doc_id', 'metadata'],
    num_rows: 185
})

In [12]:
hub = 'BroDeadlines/TEST.TDT.mini.tdt_copora_data'
index_ds_one.push_to_hub(hub)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.TDT.mini.tdt_copora_data/commit/cc5e9ad3b50fb0a955b11bcefc01869582fc39ca', commit_message='Upload dataset', commit_description='', oid='cc5e9ad3b50fb0a955b11bcefc01869582fc39ca', pr_url=None, pr_revision=None, pr_num=None)

## Upload new QA evaluate data for TDT

In [5]:
from datasets import load_dataset
main_ds = load_dataset("BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong",split="train")
main_ds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 170
})

In [3]:
import pandas as pd
from uuid import uuid4

df = pd.read_csv(f"{PATH}/departments.csv")
ques_df = pd.read_csv(f"{PATH}/mk_ques.csv")
df.head(5)

,ID,department,tel,email,website,feature
0,1,Phòng Công tác Học sinh – Sinh viên (A0003),(028) 37755054,phongctct-hssv@tdtu.edu.vn,https://student.tdtu.edu.vn,"- Quản lý dữ liệu sinh viên, cấp Thẻ sinh viên..."
1,2,Phòng Tài chính (A0007),(028) 37755070,phongkhtc@tdtu.edu.vn,NaN,- Thông báo học phí và giải đáp các thắc mắc l...
2,3,Phòng Khảo thí và Kiểm định chất lượng (A0302),(028) 37755042,dtqa@tdtu.edu.vn,NaN,"- Phản ánh các vấn đề về đề thi, giảng viên.\n..."
3,4,Phòng Đại học (A0005),(028) 37 755 051 - (028) 37755052,phongdaihoc@tdtu.edu.vn,https://undergrad.tdtu.edu.vn,"- Tổ chức và xử lý các vấn đề đăng ký KHHT, đă..."
4,5,Phòng Y tế (Nhà thi đấu - Phòng 104),(028) 22 137 081,NaN,NaN,\n- Cung cấp dịch vụ sơ cứu và chăm sóc sức kh...


In [10]:
IDS = [uuid4() for i in range(df.shape[0])]
len(IDS)

15

In [12]:
import json


a = {"isCloned": True}

def build_phone(row):
    ques = "Số điện thoại của {department} là gì?".format(department=row['department'])
    ans = "{tel}".format(tel=row['tel'])
    url = row['website']
    if not url:
        url = ""
    return {"question": ques, "answer": ans, "url": url, "group": "hỏi về các phòng chức năng", "doc_id": row['doc_id'], "metadata": json.dumps(a)}

def build_email(row):
    ques = "Email của {department} là gì?".format(department=row['department'])
    ans = "{email}".format(email=row['email'])
    url = row['website']
    if not url:
        url = ""
    return {"question": ques, "answer": ans, "url": url, "group": "hỏi về các phòng chức năng", "doc_id": row['doc_id'], "metadata": json.dumps(a)}

def build_feature(row):
    ques = "Chức năng của {department} là gì?".format(department=row['department'])
    ans = "{department} được mô tả như sau\n{feature}".format(department=row['department'], feature=row['feature'])
    url = row['website']
    if not url:
        url = ""
    return {"question": ques, "answer": ans, "url": url, "group": "hỏi về các phòng chức năng", "doc_id": row['doc_id'], "metadata": ""}

In [13]:
df

,ID,department,tel,email,website,feature
0,1,Phòng Công tác Học sinh – Sinh viên (A0003),(028) 37755054,phongctct-hssv@tdtu.edu.vn,https://student.tdtu.edu.vn,"- Quản lý dữ liệu sinh viên, cấp Thẻ sinh viên..."
1,2,Phòng Tài chính (A0007),(028) 37755070,phongkhtc@tdtu.edu.vn,NaN,- Thông báo học phí và giải đáp các thắc mắc l...
2,3,Phòng Khảo thí và Kiểm định chất lượng (A0302),(028) 37755042,dtqa@tdtu.edu.vn,NaN,"- Phản ánh các vấn đề về đề thi, giảng viên.\n..."
3,4,Phòng Đại học (A0005),(028) 37 755 051 - (028) 37755052,phongdaihoc@tdtu.edu.vn,https://undergrad.tdtu.edu.vn,"- Tổ chức và xử lý các vấn đề đăng ký KHHT, đă..."
4,5,Phòng Y tế (Nhà thi đấu - Phòng 104),(028) 22 137 081,NaN,NaN,\n- Cung cấp dịch vụ sơ cứu và chăm sóc sức kh...
5,6,Thư viện Truyền cảm hứng (Tòa nhà G),(028) 37 755 057,library@tdtu.edu.vn,lib.tdtu.edu.vn,\n- Tiếp nhận yêu cầu và/hoặc giải đáp các thắ...
6,7a,Văn phòng Tư vấn và Hỗ trợ người học (E0001) -...,(028) 37 775 015,thamvantamly@tdtu.edu.vn,https://tuvanhocduong.tdtu.edu.vn,\n- Tiếp nhận hỗ trợ các khó khăn về tâm lý/ ...
7,7b,Văn phòng Tư vấn và Hỗ trợ người học (E0001) -...,(028) 22477215,tuvanhocduong@tdtu.edu.vn,https://tuvanhocduong.tdtu.edu.vn,\n- Tư vấn học vụ (quy chế/quy định liên quan ...
8,8,Văn phòng Đoàn - Hội (A0004),(028) 37755027,vanphongdoanhoi@tdtu.edu.vn,https://vanphongdoanhoi.tdtu.edu.vn,- Nhận và trả sổ Đoàn.\n- Tiếp nhận và xử lý c...
9,9,Ký túc xá (I0005),(028) 37 760 652,ktx@tdtu.edu.vn,https://dormitory.tdtu.edu.vn,- Tiếp nhận và giải quyết các thủ tục tại Ký t...


In [14]:
data = []
for index, row in df.iterrows():
    row['doc_id'] = str(IDS.pop())
    r = build_phone(row)
    data.append(r)
    r = build_email(row)
    data.append(r)
    r = build_feature(row)
    data.append(r)

In [16]:
main_ds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 170
})

In [28]:
from datasets import concatenate_datasets, Dataset
import pandas as pd

ndf = pd.DataFrame(data)
# nds = Dataset.from_list(data)
ndf = ndf.fillna("")

In [35]:
ndf['doc_id'] = ndf['doc_id'].apply(lambda e: str(e))

In [37]:
nds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 45
})

In [40]:
fin_ds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 215
})

In [39]:
fin_ds = concatenate_datasets([main_ds, nds])
fin_ds.push_to_hub("BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong", split="train")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/786 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong/commit/339a4ccad552cd75f86aa5daa2412a2abcda7f93', commit_message='Upload dataset', commit_description='', oid='339a4ccad552cd75f86aa5daa2412a2abcda7f93', pr_url=None, pr_revision=None, pr_num=None)

## Crawl QA data

In [1]:
import requests
from bs4 import BeautifulSoup

def download_and_process(url):
  """Downloads the HTML content from the given URL and processes it using BeautifulSoup.

  Args:
    url: The URL of the webpage to download.

  Returns:
    A BeautifulSoup object representing the parsed HTML content.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for error HTTP statuses
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup
  except requests.exceptions.RequestException as e:
    print(f"Error downloading HTML: {e}")
  except Exception as e:
    print(f"Error processing HTML: {e}")
  return None

# Example usage:
url = "http://tuyensinh.hub.edu.vn/he-dao-tao/cau-hoi-tuyen-sinh-2023-106.html"
soup = download_and_process(url)

In [4]:
title = soup.title.string
title

'Câu hỏi tuyển sinh 2024'

In [16]:
title = soup.select(".thongbao_tieude")[0]
content = soup.select(".thongbao_text")[0]
# content.text

In [18]:
txts = content.text.split("?")
len(txts)

59

In [43]:
import re
pattern = r"Câu| .\d.*\?"
str = content.text
a = re.split(pattern, str)
len(a)

62

In [44]:
a[0]

'\n\nTRƯỜNG ĐẠI HỌC NGÂN HÀNG TP.HCM\n\nPHÒNG TƯ VẤN TUYỂN SINH VÀ PHÁT TRIỂN THƯƠNG HIỆU\n\r\n\t\xa0\n\nCÁC CÂU HỎI TUYỂN SINH ĐẠI HỌC CHÍNH QUY NĂM 2023 THƯỜNG GẶP\n\r\n\t\xa0\n\n\n1. Trường Đại học Ngân hàng TP.HCM là đại học công lập hay đại học dân lập? Đã được thành lập lâu chưa ?\xa0\n\n\r\n\t\t\t- Trường ĐH Ngân hàng TP.HCM là trường ĐH công lập, trực thuộc Ngân hàng nhà nước Việt Nam. Trường Đại học Ngân hàng TP.HCM chính thức được thành lập ngày 16/12/1976. Trải qua nhiều giai đoạn phát triển với những tên gọi khác nhau, ngày 20/8/2003, Trường chính thức lấy tên gọi là Trường Đại học Ngân hàng Tp. Hồ Chí Minh theo Quyết định số 174/2003/QĐ-TTg của Thủ tướng Chính phủ.\xa0\xa0\n\r\n\t\t\t- Với lịch sử gần 50năm hình thành và phát triển, Trường ngày càng khẳng định được vị thế của một trong những trường đại học đa ngành hàng đầu về đào tạo, nghiên cứu khoa học, chuyển giao công nghệ trong lĩnh vực kinh tế, tài chính- ngân hàng, kinh doanh, quản lý, luật, ngôn ngữ, công n